In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import platform

In [11]:
class MLP(nn.Module):
    """A simple MLP for binary classification."""
    def __init__(self, input_dim, hidden_dim1=512, hidden_dim2=256, dropout=0.5):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 1) # Output is a single logit
        )

    def forward(self, x):
        return self.network(x)

In [12]:
class FeatureDataset(Dataset):
    """
    A PyTorch dataset to load pre-extracted features.
    This version reads from metadata to be more robust.
    """
    def __init__(self, domains, split='train'):
        self.domains = domains
        self.features = []
        self.labels = []

        metadata_path = 'ffpp_metadata.csv'
        if not os.path.exists(metadata_path):
            raise FileNotFoundError(f"Metadata file not found at {metadata_path}. Please run download_and_prepare_ffpp.py first.")
        
        metadata = pd.read_csv(metadata_path)
        
        # Simple split for demonstration. For a real project, use a more robust split.
        train_df = metadata.sample(frac=0.8, random_state=42)
        val_df = metadata.drop(train_df.index)
        
        df = train_df if split == 'train' else val_df

        print(f"Loading {split} dataset with {len(df)} videos...")

        for index, row in tqdm(df.iterrows(), total=len(df), desc=f"Loading {split} features"):
            label = row['label']
            video_id = row['video_id']
            
            # Directory where preprocessed faces for this video are stored
            faces_dir = os.path.join('preprocessed_faces', label, video_id)
            if not os.path.exists(faces_dir):
                continue
            
            # Find all the frame images that were preprocessed
            for image_file in os.listdir(faces_dir):
                if image_file.endswith('.png'):
                    # Check if the corresponding feature file exists for all requested domains
                    feature_paths_exist = True
                    feature_paths_for_frame = {}

                    for domain in self.domains:
                        feature_filename = image_file.replace('.png', '.npy')
                        # Prefer nested path: extracted_features/<domain>/<label>/<video_id>/<frame>.npy
                        nested_feature_path = os.path.join('extracted_features', domain, label, video_id, feature_filename)
                        # Fallback to flat path if project was extracted that way
                        flat_feature_path = os.path.join('extracted_features', domain, label, feature_filename)
                        feature_path = nested_feature_path if os.path.exists(nested_feature_path) else flat_feature_path
                        
                        if not os.path.exists(feature_path):
                            feature_paths_exist = False
                            break
                        feature_paths_for_frame[domain] = feature_path
                    
                    if feature_paths_exist:
                        self.features.append(feature_paths_for_frame)
                        self.labels.append(1 if label == 'fake' else 0)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature_paths = self.features[idx]
        
        # Load and concatenate features from all specified domains
        loaded_features = [np.load(feature_paths[domain]) for domain in self.domains]
        combined_features = np.concatenate(loaded_features).astype(np.float32)
        
        label = np.float32(self.labels[idx])
        
        return torch.from_numpy(combined_features), torch.tensor(label)

In [13]:
# Set hyperparameters directly (no argparse)
default_epochs = 10
default_batch_size = 128
default_lr = 1e-4

# Model configurations for different domains
configs = {
    'Spatial': {
        'domains': ['spatial'],
        'epochs': 15,
        'batch_size': 64,
        'learning_rate': 2e-4,
        'hidden_dim1': 1024,
        'hidden_dim2': 512,
        'dropout': 0.3
    },
    'Frequency': {
        'domains': ['frequency'],
        'epochs': 20,
        'batch_size': 256,
        'learning_rate': 5e-4,
        'hidden_dim1': 128,
        'hidden_dim2': 64,
        'dropout': 0.1
    },
    'Semantic': {
        'domains': ['semantic'],
        'epochs': 12,
        'batch_size': 32,
        'learning_rate': 1e-4,
        'hidden_dim1': 512,
        'hidden_dim2': 256,
        'dropout': 0.4
    },
    'Fused (All)': {
        'domains': ['spatial', 'frequency', 'semantic'],
        'epochs': 10,
        'batch_size': 128,
        'learning_rate': 1e-4,
        'hidden_dim1': 512,
        'hidden_dim2': 256,
        'dropout': 0.5
    }
}

In [14]:
def run_pipeline(domains, epochs, batch_size, learning_rate, hidden_dim1=512, hidden_dim2=256, dropout=0.5):
    """
    Trains and evaluates a model for a given set of feature domains.
    Optimized for GPU (RTX 3060) with automatic fallback to CPU.
    """
    print(f"\n--- Running Pipeline for Domain(s): {', '.join(domains)} ---")
    print(f"Hyperparameters: epochs={epochs}, batch_size={batch_size}, lr={learning_rate}")
    print(f"Model architecture: hidden_dims=({hidden_dim1}, {hidden_dim2}), dropout={dropout}")

    # GPU setup and status - Force NVIDIA GPU (not AMD integrated graphics)
    use_cuda = torch.cuda.is_available()
    
    if use_cuda:
        # Explicitly set NVIDIA GPU (GPU 1 in your system)
        torch.cuda.set_device(0)  # CUDA device 0 = NVIDIA RTX 3060
        device = torch.device('cuda:0')
        
        print(f"✓ Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"  - GPU Index: cuda:0 (NVIDIA RTX 3060 Laptop GPU)")
        print(f"  - CUDA Version: {torch.version.cuda}")
        print(f"  - GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        print(f"  - Current GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
        
        # Enable cuDNN autotuner for better performance
        torch.backends.cudnn.benchmark = True
    else:
        device = torch.device('cpu')
        print("⚠ GPU not available, using CPU")

    # Prepare datasets
    train_dataset = FeatureDataset(domains=domains, split='train')
    val_dataset = FeatureDataset(domains=domains, split='val')

    if len(train_dataset) == 0 or len(val_dataset) == 0:
        print("\nERROR: Dataset is empty.")
        print("Please ensure feature extraction was successful and that the 'extracted_features' directory is not empty.")
        return None

    # DataLoader settings optimized for GPU
    is_windows = platform.system() == 'Windows'
    num_workers = 0 if is_windows else 4
    pin_memory = use_cuda  # Enable for faster CPU->GPU transfer
    persistent_workers = num_workers > 0

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=pin_memory,
        persistent_workers=persistent_workers
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory,
        persistent_workers=persistent_workers
    )

    # Define model, loss, optimizer
    input_dim = train_dataset[0][0].shape[0]
    model = MLP(input_dim=input_dim, hidden_dim1=hidden_dim1, hidden_dim2=hidden_dim2, dropout=dropout)
    model.to(device)
    
    if use_cuda:
        print(f"✓ Model loaded on GPU with {sum(p.numel() for p in model.parameters())} parameters")
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]"):
            features, labels = features.to(device), labels.to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader):.4f}")

    # Evaluation
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]"):
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten() >= 0.5
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Validation Metrics: Acc: {accuracy:.4f}, Prec: {precision:.4f}, Rec: {recall:.4f}, F1: {f1:.4f}")
    
    return {'Domain': ' + '.join(domains), 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}

In [15]:
results = []

for name, config in configs.items():
    result = run_pipeline(
        domains=config['domains'],
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        hidden_dim1=config['hidden_dim1'],
        hidden_dim2=config['hidden_dim2'],
        dropout=config['dropout']
    )
    if result:
        results.append(result)

# --- Print Final Comparison Table ---
if results:
    print("\n\n--- Final Performance Comparison ---")
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))


--- Running Pipeline for Domain(s): spatial ---
Hyperparameters: epochs=15, batch_size=64, lr=0.0002
Model architecture: hidden_dims=(1024, 512), dropout=0.3
✓ Using GPU: NVIDIA GeForce RTX 3060 Laptop GPU
  - GPU Index: cuda:0 (NVIDIA RTX 3060 Laptop GPU)
  - CUDA Version: 12.1
  - GPU Memory: 6.00 GB
  - Current GPU Memory Allocated: 16.25 MB
Loading train dataset with 280 videos...


Loading train features: 100%|██████████| 280/280 [00:00<00:00, 718.40it/s]


Loading val dataset with 70 videos...


Loading val features: 100%|██████████| 70/70 [00:00<00:00, 764.35it/s]



✓ Model loaded on GPU with 2623489 parameters


...

Epoch 9, Training Loss: 0.3055


Epoch 10/10 [Training]: 100%|██████████| 44/44 [00:03<00:00, 11.47it/s]


Epoch 10, Training Loss: 0.2954


Epoch 10/10 [Validation]: 100%|██████████| 11/11 [00:00<00:00, 13.24it/s]

Validation Metrics: Acc: 0.8670, Prec: 0.8835, Rec: 0.9790, F1: 0.9288


--- Final Performance Comparison ---
                        Domain  Accuracy  Precision   Recall       F1
                       spatial  0.754825   0.869637 0.850686 0.860057
                     frequency  0.885633   0.885633 1.000000 0.939348
                      semantic  0.774839   0.871981 0.874092 0.873035
spatial + frequency + semantic  0.867048   0.883467 0.979015 0.928790
